In [1]:
import cloudknot as ck
ck.set_region('us-west-2')

In [3]:
def afq_cam_can(subject):
    import AFQ.data as afqd
    import AFQ.api as api
    import os.path as op
    import s3fs
    
    local_bids_folder = "./cam_can"
    bucket = "cam-can.afq-insight"
    s3_prefix = "raw/cc700/mri/pipeline/release004/BIDS_NRDG/BIDS_NRDG"
    
    study = afqd.S3BIDSStudy(
        "Cam-CAN",
        bucket,
        s3_prefix,
        subjects=10,
        anon=False,
        random_seed=42)
    study.download(local_bids_folder)
    
    myafq = api.AFQ(local_bids_folder, virtual_frame_buffer=True)
    myafq.export_all()

    s3fs.S3FileSystem().put(
        op.join(local_bids_folder, "afq"),
        op.join(bucket, s3_prefix, "afq"),
        recursive=True)
    

In [ ]:
afq_knot = ck.Knot(name='afq_cam_can-64gb-20804-0',
                  func=afq_cam_can,
                  base_image='libglxvfb:1',
                  image_github_installs="https://github.com/richford/pyAFQ.git@s3-bids-fetch",
                  pars_policies=('AmazonS3FullAccess',),
                  resource_type="SPOT",
                  bid_percentage=100,
                  memory=64000)